In [1]:
import kmapper as km
import pandas as pd
import sklearn
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from matplotlib import pyplot as plt

from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn import model_selection
from sklearn.dummy import DummyClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import xgboost as xgb

pd.set_option("display.max_columns", 100)

/Users/roryduthie/.pyenv/versions/3.8.1/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
transactions_df = pd.read_csv('trimmed_transaction_dataset.csv')

In [5]:
transactions_df = transactions_df.drop(['Address', 'Index', 'Unnamed: 0', ' ERC20_most_rec_token_type', ' ERC20 most sent token type'], axis=1)

In [6]:
from sklearn import ensemble, cluster

In [7]:
features = ['Avg min between received tnx', 'Time Diff between first and last (Mins)', 'Received Tnx', 'Number of Created Contracts', 'Unique Received From Addresses', 'avg val received', 'min val sent', 'total transactions (including tnx to create contract', 'total ether received', ' Total ERC20 tnxs', ' ERC20 total Ether received', ' ERC20 total ether sent', ' ERC20 uniq rec addr', ' ERC20 min val rec', ' ERC20 min val sent', ' ERC20 uniq sent token name']
X = np.array(transactions_df[features])
y = np.array(transactions_df.FLAG)

projector = ensemble.IsolationForest(random_state=0, n_jobs=-1)
projector.fit(X)
lens1 = projector.decision_function(X)

mapper = km.KeplerMapper(verbose=3)
lens2 = mapper.fit_transform(X, projection="knn_distance_5")

lens = np.c_[lens1, lens2]

KeplerMapper(verbose=3)
..Composing projection pipeline of length 1:
	Projections: knn_distance_5
	Distance matrices: False
	Scalers: MinMaxScaler()
..Projecting on data shaped (8981, 16)

..Projecting data using: knn_distance_5

..Scaling with: MinMaxScaler()



In [44]:
G = mapper.map(
    lens,
    X,
    cover = km.Cover(n_cubes=30,
                     perc_overlap=.15),
    clusterer=sklearn.cluster.KMeans(n_clusters=20,random_state=3471))

print(f"num nodes: {len(G['nodes'])}")
print(f"num edges: {sum([len(values) for key, values in G['links'].items()])}")

Mapping on data shaped (8981, 16) using lens shaped (8981, 2)

Minimal points in hypercube before clustering: 20
Creating 900 hypercubes.
Cube_0 is empty.

Cube_1 is empty.

Cube_2 is empty.

Cube_3 is empty.

Cube_4 is empty.

Cube_5 is empty.

Cube_6 is empty.

Cube_7 is empty.

   > Found 20 clusters in hypercube 8.
Cube_9 is empty.

Cube_10 is empty.

Cube_11 is empty.

Cube_12 is empty.

Cube_13 is empty.

   > Found 20 clusters in hypercube 14.
   > Found 20 clusters in hypercube 15.
   > Found 20 clusters in hypercube 16.
   > Found 20 clusters in hypercube 17.
   > Found 20 clusters in hypercube 18.
   > Found 20 clusters in hypercube 19.
   > Found 20 clusters in hypercube 20.
   > Found 20 clusters in hypercube 21.
   > Found 20 clusters in hypercube 22.
   > Found 20 clusters in hypercube 23.
   > Found 20 clusters in hypercube 24.
   > Found 20 clusters in hypercube 25.
   > Found 20 clusters in hypercube 26.
   > Found 20 clusters in hypercube 27.
   > Found 20 clusters in

In [45]:
_ = mapper.visualize(
    G,
    custom_tooltips=y,
    color_values=y,
    color_function_name="target",
    path_html="fraud_data.html",
    X=X,
    X_names=list(transactions_df[features].columns),
    lens=lens,
    lens_names=["IsolationForest", "KNN-distance 5"],
    title="Detecting encrypted Tor Traffic with Isolation Forest and Nearest Neighbor Distance"
)

Wrote visualization to: fraud_data.html


In [46]:
G

{'nodes': defaultdict(list,
             {'cube8_cluster0': [7259],
              'cube8_cluster1': [6142],
              'cube8_cluster2': [2395],
              'cube8_cluster3': [1251],
              'cube8_cluster4': [6315],
              'cube8_cluster5': [1981],
              'cube8_cluster6': [5212, 5364],
              'cube8_cluster7': [5783],
              'cube8_cluster8': [2549],
              'cube8_cluster9': [1162],
              'cube8_cluster10': [5642],
              'cube8_cluster11': [5124],
              'cube8_cluster12': [6527],
              'cube8_cluster13': [2754],
              'cube8_cluster14': [3619],
              'cube8_cluster15': [6740],
              'cube8_cluster16': [1873],
              'cube8_cluster17': [3835],
              'cube8_cluster18': [6379],
              'cube8_cluster19': [7530],
              'cube14_cluster0': [469, 725, 1183, 3958, 5292],
              'cube14_cluster1': [1513],
              'cube14_cluster2': [6053],
           

In [55]:
val = 8635
for rows in G['nodes']:
    row_list = G['nodes'][rows]
    if val in row_list:
        print(rows)
        print(G['nodes'][rows])

cube18_cluster9
[8221, 8340, 8635]


In [68]:
val = 8865
for rows in G['nodes']:
    row_list = G['nodes'][rows]
    if val in row_list:
        print(rows)
        print(G['nodes'][rows])

cube28_cluster0
[9, 68, 86, 93, 98, 102, 156, 175, 190, 195, 240, 278, 328, 428, 449, 456, 517, 520, 534, 644, 782, 796, 797, 816, 850, 851, 872, 948, 1042, 1082, 1132, 1139, 1175, 1200, 1212, 1258, 1274, 1279, 1289, 1298, 1335, 1464, 1468, 1471, 1551, 1590, 1597, 1604, 1612, 1632, 1671, 1711, 1729, 1790, 1795, 1829, 1849, 1860, 1898, 1901, 1908, 1915, 2007, 2047, 2125, 2199, 2279, 2354, 2460, 2600, 2616, 2627, 2653, 2676, 2678, 2708, 2726, 2735, 2764, 2863, 2904, 2929, 2951, 2955, 2956, 2998, 3027, 3032, 3066, 3076, 3080, 3112, 3154, 3167, 3285, 3325, 3364, 3394, 3423, 3433, 3440, 3447, 3563, 3600, 3602, 3627, 3659, 3661, 3667, 3675, 3722, 3738, 3756, 3804, 3805, 3870, 3892, 3905, 3922, 3929, 3932, 3948, 4003, 4036, 4119, 4134, 4302, 4311, 4318, 4396, 4402, 4450, 4454, 4569, 4592, 4647, 4713, 4747, 4773, 4838, 4839, 4885, 4892, 4929, 4942, 5057, 5060, 5070, 5094, 5196, 5226, 5314, 5385, 5386, 5517, 5526, 5541, 5604, 5610, 5635, 5748, 5801, 5862, 5949, 5989, 6008, 6021, 6030, 6139, 617